In [1]:
import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p2/ch4/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p2/ch4/test.csv')

train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,NCR06,12.500,Low Fat,0.006760,Household,42.8112,OUT013,1987,High,Tier 3,Supermarket Type1,639.1680
1,FDW11,12.600,Low Fat,0.048741,Breads,60.4194,OUT013,1987,High,Tier 3,Supermarket Type1,990.7104
2,FDH32,12.800,Low Fat,0.075997,Fruits and Vegetables,97.1410,OUT013,1987,High,Tier 3,Supermarket Type1,2799.6890
3,FDL52,6.635,Regular,0.046351,Frozen Foods,37.4506,OUT017,2007,NaN,Tier 2,Supermarket Type1,1176.4686
4,FDO09,13.500,Regular,0.125170,Snack Foods,261.4910,OUT013,1987,High,Tier 3,Supermarket Type1,3418.8830


# EDA

In [2]:
train.shape, test.shape

((6818, 12), (1705, 11))

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6818 non-null   object 
 1   Item_Weight                5656 non-null   float64
 2   Item_Fat_Content           6818 non-null   object 
 3   Item_Visibility            6818 non-null   float64
 4   Item_Type                  6818 non-null   object 
 5   Item_MRP                   6818 non-null   float64
 6   Outlet_Identifier          6818 non-null   object 
 7   Outlet_Establishment_Year  6818 non-null   int64  
 8   Outlet_Size                4878 non-null   object 
 9   Outlet_Location_Type       6818 non-null   object 
 10  Outlet_Type                6818 non-null   object 
 11  Item_Outlet_Sales          6818 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 639.3+ KB


In [4]:
train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,5656.000000,6818.000000,6818.000000,6818.000000,6818.000000
mean,12.872703,0.066121,140.419533,1997.885890,2190.941459
std,4.651034,0.051383,62.067861,8.339795,1706.131256
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.785000,0.026914,93.610050,1987.000000,836.577700
50%,12.600000,0.053799,142.448300,1999.000000,1806.648300
75%,17.000000,0.095273,185.060150,2004.000000,3115.944000
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [5]:
train.describe(include='O') # unique가 많다 == 컬럼이 많다 == 원핫 인코딩은 어려움 == 레이블 인코딩 고려

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,6818,6818,6818,6818,4878,6818,6818
unique,1554,5,16,10,3,3,4
top,FDO19,Low Fat,Snack Foods,OUT046,Medium,Tier 3,Supermarket Type1
freq,9,4092,963,763,2228,2664,4474


In [6]:
test.describe(include='O')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,1705,1705,1705,1705,1235,1705,1705
unique,1077,5,16,10,3,3,4
top,DRD15,Low Fat,Fruits and Vegetables,OUT013,Medium,Tier 3,Supermarket Type1
freq,4,997,272,207,565,686,1103


In [7]:
set(train['Item_Type']) == set(test['Item_Type']) # 유니크 내용이 같은지 확인

True

In [8]:
train.isnull().sum()

Item_Identifier                 0
Item_Weight                  1162
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1940
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [9]:
test.isnull().sum()

Item_Identifier                0
Item_Weight                  301
Item_Fat_Content               0
Item_Visibility                0
Item_Type                      0
Item_MRP                       0
Outlet_Identifier              0
Outlet_Establishment_Year      0
Outlet_Size                  470
Outlet_Location_Type           0
Outlet_Type                    0
dtype: int64

In [10]:
train['Item_Type'].value_counts()

Item_Type
Snack Foods              963
Fruits and Vegetables    960
Household                728
Frozen Foods             701
Dairy                    561
Baking Goods             522
Canned                   517
Health and Hygiene       416
Meat                     350
Soft Drinks              345
Breads                   190
Hard Drinks              171
Others                   137
Starchy Foods            115
Breakfast                 92
Seafood                   50
Name: count, dtype: int64

# 전처리

In [11]:
# train에서 Item_Outlet_Sales(y)를 가장 먼저 빼냄 추천
target = train.pop('Item_Outlet_Sales') 
target

0        639.1680
1        990.7104
2       2799.6890
3       1176.4686
4       3418.8830
          ...    
6813    3615.2940
6814     923.4646
6815    2530.7058
6816     818.9340
6817    1973.4312
Name: Item_Outlet_Sales, Length: 6818, dtype: float64

In [12]:
train.isnull().sum()

Item_Identifier                 0
Item_Weight                  1162
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1940
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [13]:
# 최빈값으로 결측치 처리(.min()/.mean() 등 결과 잘 나오는 걸로 결정)
train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].mode()[0])
train['Outlet_Size'] = train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])

In [14]:
train.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

# 인코딩

In [15]:
cols = train.columns[train.dtypes == object]
cols

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [16]:
print(train.shape, test.shape)
df = pd.concat([train, test]) # 한번에 같은 기준으로 인코딩 하기 위해 합침
print(df.shape)

(6818, 11) (1705, 11)
(8523, 11)


In [17]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() # 아이템 아이디가 1500개라 레이블 인코딩이 적합

for col in cols:
    df[col] = le.fit_transform(df[col])
    
df

# 원핫 인코딩
# df = df.drop('Item_Identifier', axis=1) 종류가 너무 많으면 삭제도 방법(학습시간 1분 넘어가면 안됨)
# df = pd.get_dummies(df)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,1467,12.500,1,0.006760,9,42.8112,1,1987,0,2,1
1,1075,12.600,1,0.048741,1,60.4194,1,1987,0,2,1
2,450,12.800,1,0.075997,6,97.1410,1,1987,0,2,1
3,613,6.635,2,0.046351,5,37.4506,2,2007,1,1,1
4,695,13.500,2,0.125170,13,261.4910,1,1987,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
1700,1502,NaN,1,0.097769,9,142.4496,4,1985,2,0,0
1701,1168,NaN,2,0.153457,4,264.0910,4,1985,2,0,0
1702,474,20.750,2,0.129864,12,240.1880,3,2009,1,2,2
1703,69,14.650,1,0.171282,7,54.0614,2,2007,3,1,1


In [18]:
train = df.iloc[:len(train)].copy()
test = df.iloc[len(train):].copy()

train.shape, test.shape

((6818, 11), (1705, 11))

In [19]:
from sklearn.model_selection import train_test_split

# X : 문제, y : 정답 / train : 학습용, val : 평가용
X_train, X_val, y_train, y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((5454, 11), (1364, 11), (5454,), (1364,))

In [20]:
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [21]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [22]:
pred = lr.predict(X_val)
pred

array([3013.6389915 , 1279.9911607 , 1599.52850742, ..., 2336.31109686,
       3998.52317113, 3524.18489506], shape=(1364,))

In [23]:
root_mean_squared_error(y_val, pred)

1124.1806246657081

In [24]:
# 앞 뒤 바뀌어도 상관없음
root_mean_squared_error(pred, y_val)

1124.1806246657081

In [25]:
mean_squared_error(y_val, pred)

1263782.0768737819

In [26]:
mean_absolute_error(y_val, pred)

860.4475391298243

In [27]:
r2_score(y_val, pred)

0.5131899691872113

In [28]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=0)
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [29]:
pred = rf.predict(X_val)
pred

array([2935.159326,  886.792336, 2079.772776, ..., 1721.539086,
       4845.09318 , 4788.160622], shape=(1364,))

In [30]:
root_mean_squared_error(y_val, pred)

1054.3067776418388

In [31]:
mean_squared_error(y_val, pred)

1111562.7813815176

In [32]:
mean_absolute_error(y_val, pred)

747.2553289824048

In [33]:
r2_score(y_val, pred)

0.5718249833125864

In [34]:
import lightgbm as lgb

model = lgb.LGBMRegressor(random_state=0, verbose=-1)
model.fit(X_train, y_train)

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [35]:
pred = model.predict(X_val)
pred

array([2784.17570849, 1278.07452383, 2102.05166055, ..., 2070.01790736,
       4786.63604236, 3509.95852465], shape=(1364,))

In [36]:
root_mean_squared_error(y_val, pred)

1056.1320427013504

In [37]:
mean_squared_error(y_val, pred)

1115414.891620527

In [38]:
mean_absolute_error(y_val, pred)

736.3567118383717

In [39]:
r2_score(y_val, pred)

0.5703411468676314

# 예측 파일생성

In [40]:
result = model.predict(test)
result

array([1229.28965525,  805.67309748, 1680.06702167, ..., 3465.68405885,
        866.82924777, 1338.44413614], shape=(1705,))

In [41]:
submit = pd.DataFrame({'pred':result})
submit.to_csv('result.csv', index=False)

In [ ]:
# 1. 컬럼명과 1개만 있는지 확인 2. row수가 test 데이터의 양과 같은지 확인
pd.read_csv('result.csv')

,pred
0,1229.289655
1,805.673097
2,1680.067022
3,1534.611983
4,2903.195812
...,...
1700,341.425045
1701,910.334267
1702,3465.684059
1703,866.829248
